**Disclaimer**:

Это не соревнование с Каггла, я согласовал это с Александром Владимировичем: соревнование - Sibur Challenge 2021: прогноз спроса https://sibur.ai-community.com/competitions/5/tasks/13

Ccылка на лидерборд: https://sibur.ai-community.com/competitions/5/tasks/13/rating (49 позиция)

Весь процесс аналогичен Кагглу, тот же лидерборд итд

На нем я занял 49 место, все, что выше 110 - лучше бейзлайна

Проект состоит из трех файлов:

1) Sibur Challenge 2021.ipynb - сам анализ данных, построение и оптимизация модели

2) gd_model.sav - файл с сохраненной моделью

3) predict.py - это и есть submission file - туда загружается не сами предсказания, а функции, которые получают на вход тестовую выборку(которую участники не знают) - они делают препроцессинг и предсказания(вызывая файл с моделью). К слову, в этом плане это немного сложнее каглла, где нужен зачастую только csv файл с предсказаниями

**Disclaimer 2**:

оптимизация hyperopt у меня прогонялась около 25-30 минут + обычные модели в сумме несколько минут - поэтому скрипт быстро не прогонится

**Задача**

Задача состоит в том, чтобы на основе данных о продажах(кто(какой менеджер), куда(в какую страну и регион), кому(какой компании) в каком количестве продал). Данные имеются за несколько лет, они аггрегируются по вышеупомяннутым колонкам(по условию организаторов), затем строится модель, которая предсказывает спрос на заданный месяц

**Формилировка с сайта**

СИБУР производит и продает различную нефтехимическую продукцию во многие страны мира. Поэтому прогноз спроса необходимо строить с детализацией до продукта, региона, клиента и менеджера, осуществляющего сделку.

Для построения модели даны сделки за тренировочный период (с 2018-01-01 по 2020-07-31) с полями, описывающими сделку. Тестовый набор данных соответствует периоду с 2020-08-01 по 2021-07-31.


Поля в наборе данных:

material_code - код продукта,

company_code - код клиента, который всегда равен 0 для спотовых сделок,

country - страна, в которую осуществляется продажа,

region - регион внутри страны, в которую осуществляется продажа; для большинства стран не детализирован,

manager_code - код менеджера, ведущего сделку,

month - месяц сделки,

material_lvl1_name, material_lvl2_name, material_lvl3_name - группировка продукта по категориям разных уровней,

contract_type - тип сделки,

date - точная дата сделки,

volume - объем сделки.

Структура кода и валидации

В этом соревновании вам предстоит загружать код, который вычисляет предсказания, т. е. суммарный объем для следующего месяца для 
каждой группы material_code, company_code, country, region, manager_code.


в наборе данных есть 941 группа material_code, company_code, country, region, manager_code, при этом нет групп, которые встречаются только в тестовом периоде,
полностью сформированный набор предсказаний содержит, таким образом, 941 строку, для каждой из которых вычислены 12 значений суммарного объема для каждого месяца в тестовом периоде,
код решения должен принимать на вход целевой месяц и сделки, предшествующие ему (с начала тренировочного периода) и возвращать предсказания объемов для целевого месяца; 

In [4]:
import pandas as pd 
import numpy as np

In [5]:
df_start = pd.read_csv('/Users/polyanaboss/Desktop/sc2021_train_deals.csv',  parse_dates=["month", "date"])

In [6]:
df_start.head()

,material_code,company_code,country,region,manager_code,month,material_lvl1_name,material_lvl2_name,material_lvl3_name,contract_type,date,volume
0,134,0,Литва,Литва,12261,2018-01-01,Базовые полимеры,ПЭ,ПЭНП,Спот,2018-01-01,43.0
1,197,0,Китай,Китай,16350,2018-01-01,Базовые полимеры,ПЭ,ПЭНП,Спот,2018-01-02,95.0
2,794,2162,Казахстан,Атырауская обл.,10942,2018-01-01,Базовые полимеры,ПП,ПП,Контракт,2018-01-02,57.0
3,134,0,Литва,Литва,12261,2018-01-01,Базовые полимеры,ПЭ,ПЭНП,Спот,2018-01-02,21.0
4,133,0,Китай,Китай,17745,2018-01-01,Базовые полимеры,ПЭ,ПЭНП,Спот,2018-01-02,150.0


In [7]:
#колонки для аггрегации(это условие соревнования)
AGG_COLS = ["material_code", "company_code", "country", "region", "manager_code"]

In [8]:
df = df_start.groupby(AGG_COLS + ["month"])["volume"].sum().unstack(fill_value=0)

In [9]:
#наши данные в виде сводной таблицы для материала, компании, страны-экспортера, региона и менеджера, который делал продажи
#далее - идут сами продажи по месяцам
df.head()

month                                                                2018-01-01  \
material_code company_code country    region           manager_code               
124           7278         Россия     Респ. Татарстан  17460              340.0   
133           0            Белоруссия Минская обл.     10942                0.0   
                                      Могилевская обл. 10942                0.0   
                                      г. Минск         10942                0.0   
                           Казахстан  г. Нур-Султан    13301                0.0   

month                                                                2018-02-01  \
material_code company_code country    region           manager_code               
124           7278         Россия     Респ. Татарстан  17460              340.0   
133           0            Белоруссия Минская обл.     10942                0.0   
                                      Могилевская обл. 10942                0.0   
                                      г. Минск         10942               20.0   
                           Казахстан  г. Нур-Султан    13301                0.0   

month                                                                2018-03-01  \
material_code company_code country    region           manager_code               
124           7278         Россия     Респ. Татарстан  17460              260.0   
133           0            Белоруссия Минская обл.     10942                0.0   
                                      Могилевская обл. 10942                0.0   
                                      г. Минск         10942                0.0   
                           Казахстан  г. Нур-Султан    13301               30.0   

month                                                                2018-04-01  \
material_code company_code country    region           manager_code               
124           7278         Россия     Респ. Татарстан  17460              240.0   
133           0            Белоруссия Минская обл.     10942              200.0   
                                      Могилевская обл. 10942                0.0   
                                      г. Минск         10942                0.0   
                           Казахстан  г. Нур-Султан    13301               30.0   

month                                                                2018-05-01  \
material_code company_code country    region           manager_code               
124           7278         Россия     Респ. Татарстан  17460              220.0   
133           0            Белоруссия Минская обл.     10942               60.0   
                                      Могилевская обл. 10942              140.0   
                                      г. Минск         10942               40.0   
                           Казахстан  г. Нур-Султан    13301                0.0   

month                                                                2018-06-01  \
material_code company_code country    region           manager_code               
124           7278         Россия     Респ. Татарстан  17460              220.0   
133           0            Белоруссия Минская обл.     10942                0.0   
                                      Могилевская обл. 10942                0.0   
                                      г. Минск         10942              160.0   
                           Казахстан  г. Нур-Султан    13301                0.0   

month                                                                2018-07-01  \
material_code company_code country    region           manager_code               
124           7278         Россия     Респ. Татарстан  17460              220.0   
133           0            Белоруссия Минская обл.     10942                0.0   
                                      Могилевская обл. 10942                0.0   
                                      г. Минск         10942              180.0   
          

In [10]:
df.shape

(941, 31)

In [11]:
#разделяем на тест и трейн выборки по временным рядам
tr_range = pd.date_range("2019-01-01", "2020-03-01", freq="MS")
ts_range = pd.date_range("2020-04-01", "2020-07-01", freq="MS")

In [12]:
#эта функция получает данные и месяцы, и преобразует данные с заданным оффсетом(смещением) - здесь для каждого месяца 
#мы будем брать данные за предыдущие 6 месяцев, а так же еще несколько показателей - о них ниже
def get_features(df: pd.DataFrame, month: pd.Timestamp) -> pd.DataFrame:
    """Calculate features for `month`."""
    # задаем полугодовой период
    start_period = month - pd.offsets.MonthBegin(6)
    end_period = month - pd.offsets.MonthBegin(1)

    df = df.loc[:, :end_period]

    features = pd.DataFrame([], index=df.index)
    features["month"] = month.month
    #веичина спроса за каждый месяц из 6
    features[[f"vol_tm{i}" for i in range(6, 0, -1)]] = df.loc[:, start_period:end_period].copy()
    #так же будем использовать скользящие среднее, максимум и минимум за 12 месяцев до заданного месяца
    rolling = df.rolling(12, axis=1, min_periods=1)
    features = features.join(rolling.mean().iloc[:, -1].rename("last_year_avg"))
    features = features.join(rolling.min().iloc[:, -1].rename("last_year_min"))
    features = features.join(rolling.max().iloc[:, -1].rename("last_year_max"))
    features["month"] = month.month
    return features

In [13]:
full_features = {}
#теперь получим все данные с помощью функции
for dataset, dataset_range in zip(["tr", "ts"], [tr_range, ts_range]):
    dataset_features = []
    for target_month in dataset_range:
        features = get_features(df, target_month)
        features["target"] = df[target_month]
        dataset_features.append(features.reset_index())
    full_features[dataset] = pd.concat(dataset_features, ignore_index=True)

In [14]:
FTS_COLS = ["material_code", "company_code","manager_code", "month",
            "vol_tm6", "vol_tm5", "vol_tm4", "vol_tm3", "vol_tm2", "vol_tm1",
            "last_year_avg", "last_year_min", "last_year_max"]
TARGET = "target"

In [15]:
#для простоты напишем функцию - по условию - это RMSLE
def get_score(model):
    preds = model.predict(full_features['ts'][FTS_COLS])
    preds = np.where(preds < 0, 0,  preds)
    res = np.sqrt(mean_squared_log_error(preds, full_features['ts'][TARGET]))
    return res

In [16]:
CAT_COLS = ["material_code", "company_code", "country", "region", "manager_code", "month"]

In [17]:
from sklearn.ensemble import RandomForestRegressor

In [23]:
from keras.metrics import MeanSquaredLogarithmicError
from sklearn.metrics import mean_squared_log_error

In [19]:
FTS_COLS_no_cat = ["material_code", "company_code","manager_code", "month",
            "vol_tm6", "vol_tm5", "vol_tm4", "vol_tm3", "vol_tm2", "vol_tm1",
            "last_year_avg", "last_year_min", "last_year_max"]


In [20]:
#аналогичная функция скора только без учета категориальных переменных
def get_score_no_cat(model):
    preds = model.predict(full_features['ts'][FTS_COLS_no_cat])
    preds = np.where(preds < 0, 0,  preds)
    res = np.sqrt(mean_squared_log_error(preds, full_features['ts'][TARGET]))
    return res

In [21]:
#модель случайного леса
rf = RandomForestRegressor(n_estimators=1000, n_jobs=-1, random_state=0)
rf.fit(full_features["tr"][FTS_COLS_no_cat], full_features["tr"][TARGET])

RandomForestRegressor(n_estimators=1000, n_jobs=-1, random_state=0)

In [24]:
#скор
get_score_no_cat(rf)

2.1805620504884153

In [184]:
from sklearn.ensemble import AdaBoostRegressor, BaggingRegressor, ExtraTreesRegressor, GradientBoostingRegressor, StackingRegressor, VotingRegressor
    
    
    

In [188]:
#попробуем сразу несколько моделей с дефолтными параметрами
estimators = [
        ('rf', RandomForestRegressor()),
        ('grd', GradientBoostingRegressor())]

models = {'AdaBoostRegressor' : AdaBoostRegressor(),
          'BaggingRegressor' : BaggingRegressor(),
          'ExtraTreesRegressor' : ExtraTreesRegressor(), 
          'GradientBoostingRegressor' : GradientBoostingRegressor(),
          'StackingRegressor' : StackingRegressor(estimators = estimators), 
          'VotingRegressor' : VotingRegressor(estimators = estimators)}


In [204]:
#фитим каждую модель на наших данных и выводим принт об удачном фитинге
results = {}
for model in models.values():
    model.fit(full_features["tr"][FTS_COLS_no_cat], full_features["tr"][TARGET])
    results[model] = get_score_no_cat(model)
    print(f'{model} is fitted successfully')

AdaBoostRegressor() is fitted successfully
BaggingRegressor() is fitted successfully
ExtraTreesRegressor() is fitted successfully
GradientBoostingRegressor() is fitted successfully
StackingRegressor(estimators=[('rf', RandomForestRegressor()),
                              ('grd', GradientBoostingRegressor())]) is fitted successfully
VotingRegressor(estimators=[('rf', RandomForestRegressor()),
                            ('grd', GradientBoostingRegressor())]) is fitted successfully


In [205]:
#смотрим на результаты
results

{AdaBoostRegressor(): 4.014653725590492,
 BaggingRegressor(): 2.117072304193702,
 ExtraTreesRegressor(): 2.144555869208375,
 GradientBoostingRegressor(): 2.1428955663988773,
 StackingRegressor(estimators=[('rf', RandomForestRegressor()),
                               ('grd', GradientBoostingRegressor())]): 2.1179729183881637,
 VotingRegressor(estimators=[('rf', RandomForestRegressor()),
                             ('grd', GradientBoostingRegressor())]): 2.1387699089897034}

In [26]:
from sklearn.ensemble import RandomForestRegressor, StackingRegressor, GradientBoostingRegressor

In [27]:
#стакинг модели из градиентного бустинга и случайного леса
rf = RandomForestRegressor(n_estimators = 2000, n_jobs = -1, random_state = 0)
gd = GradientBoostingRegressor(loss ='huber', random_state=0)
estimators = [
        ('rf', RandomForestRegressor()),
        ('grd', GradientBoostingRegressor())]
stacking = StackingRegressor(estimators=estimators)

In [30]:
rf.fit(full_features["tr"][FTS_COLS_no_cat], full_features["tr"][TARGET])

RandomForestRegressor(n_estimators=2000, n_jobs=-1, random_state=0)

In [31]:
gd.fit(full_features["tr"][FTS_COLS_no_cat], full_features["tr"][TARGET])

GradientBoostingRegressor(loss='huber', random_state=0)

In [37]:
from sklearn.metrics import mean_squared_log_error

In [38]:
get_score_no_cat(rf)

2.1289519327231705

In [39]:
get_score_no_cat(gd)

1.995300247739532

In [40]:
stacking.fit(full_features["tr"][FTS_COLS_no_cat], full_features["tr"][TARGET])

StackingRegressor(estimators=[('rf', RandomForestRegressor()),
                              ('grd', GradientBoostingRegressor())])

In [41]:
get_score_no_cat(stacking)

2.1283960084226705

In [45]:
#так же попробуем алгоритм catboost, который позволяет передавать категориальыне переменные 
from catboost import CatBoostRegressor
model_cat = CatBoostRegressor(iterations=2000,
                          early_stopping_rounds=30,
                          depth=6,
                          cat_features=CAT_COLS,
                          random_state=0,
                          verbose=10)
model_cat.fit(full_features["tr"][FTS_COLS], full_features["tr"][TARGET],
          eval_set=(full_features["ts"][FTS_COLS], full_features["ts"][TARGET]))

Learning rate set to 0.048815
0:	learn: 306.0235982	test: 320.9469382	best: 320.9469382 (0)	total: 73.9ms	remaining: 2m 27s
10:	learn: 228.8491450	test: 263.9061136	best: 263.9061136 (10)	total: 172ms	remaining: 31s
20:	learn: 184.5916748	test: 236.9767402	best: 236.9767402 (20)	total: 297ms	remaining: 28s
30:	learn: 160.8291164	test: 225.2995770	best: 225.2995770 (30)	total: 384ms	remaining: 24.4s
40:	learn: 147.7530907	test: 219.9508977	best: 219.9508977 (40)	total: 468ms	remaining: 22.3s
50:	learn: 140.6362480	test: 217.7432075	best: 217.7432075 (50)	total: 556ms	remaining: 21.2s
60:	learn: 135.5530661	test: 217.1163753	best: 217.0845421 (59)	total: 634ms	remaining: 20.2s
70:	learn: 131.9383371	test: 217.0239015	best: 217.0239015 (70)	total: 728ms	remaining: 19.8s
80:	learn: 129.8237580	test: 217.2954534	best: 217.0239015 (70)	total: 839ms	remaining: 19.9s
90:	learn: 127.2426976	test: 217.4936020	best: 217.0239015 (70)	total: 938ms	remaining: 19.7s
100:	learn: 125.7029039	test: 217.

In [46]:
get_score(model_cat)

2.2891901017334026

по результатам видно, что наилучшая модель - градиентный бустинг, попробуем его оптимизировать с помощью hyperopt

In [51]:
from hyperopt import fmin, STATUS_OK, hp, tpe, Trials

In [171]:
#defining the space of parameters
space = {
        "max_depth" : hp.quniform("max_depth", 10,2000,10),
        "min_samples_leaf" : hp.uniform("min_samples_leaf", 0,0.5),
        "min_samples_split" : hp.uniform("min_samples_split", 0,1),
        "n_estimators" : hp.choice("n_estimators", [100,200,300,400,600,800,1000,1200, 1400, 1600, 1800, 2000]),
        "loss" : hp.choice("loss",["ls", "lad", "huber", "quantile"]),
        "learning_rate" :  hp.uniform("learning_rate", 0,1)}

In [172]:
#задаем функцию тьюининга
def objective(space):
    model = GradientBoostingRegressor(
                                  max_depth = space["max_depth"],
                                  min_samples_leaf = space["min_samples_leaf"],
                                  min_samples_split = space["min_samples_split"],
                                  n_estimators = space["n_estimators"],
                                  loss = space['loss'],
                                  learning_rate = space['learning_rate']
                                  )
    model.fit(full_features["tr"][FTS_COLS_no_cat], full_features["tr"][TARGET])
    
    return {"loss" : get_score_no_cat(model), "status" : STATUS_OK}

In [173]:
#сама оптимизация - около получаса
trials = Trials()

optimized_gd = fmin(fn = objective,
                   space = space, 
                   algo = tpe.suggest,
                   max_evals = 300, 
                   trials = trials)

 23%|██▎       | 70/300 [14:10<46:34, 12.15s/trial, best loss: 1.8633872204793933]  


KeyboardInterrupt: 

In [60]:
#смотрим на выбранные гиперпараметры
optimized_gd

{'learning_rate': 0.022919175228008782,
 'loss': 1,
 'max_depth': 860.0,
 'min_samples_leaf': 0.0616290956659412,
 'min_samples_split': 0.1870747466430237,
 'n_estimators': 1}

In [181]:
#строим модель бустинга по тем параметрам, которые нащел hyperopt
gd_optimized = GradientBoostingRegressor(learning_rate=0.02, loss = 'lad', max_depth=860, min_samples_leaf=0.06,
                                        min_samples_split=0.19, n_estimators=200)

In [182]:
#fitting
gd_optimized.fit(full_features["tr"][FTS_COLS_no_cat], full_features["tr"][TARGET])

GradientBoostingRegressor(learning_rate=0.02, loss='lad', max_depth=860,
                          min_samples_leaf=0.06, min_samples_split=0.19,
                          n_estimators=200)

In [183]:
#скор
get_score_no_cat(gd_optimized)

1.8584706374016846

In [153]:
#сохраняем модель с помощью библиотеки pickle
import pickle

In [156]:
pickle.dump(gd_optimized, open('gd_model.sav', 'wb'))